In [1]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import time
from rembg import remove 
from PIL import Image, ImageFont, ImageDraw

# Initialize webcam
webcam = cv2.VideoCapture(0)

# Define the region of interest
region_top = 40
region_bottom = 314
region_left = 10
region_right = 314

# Load your pre-trained model
model2 = load_model("newdata.h5")

# Define class labels
class_labels =  [ 'ક','ખ', 'ગ','ઘ', 'ચ', 'છ', 'જ', 'ટ', 'ઠ', 'ડ', 'ઢ', 'ણ', 'ત', 'થ', 'દ', 'ધ', 'ન',
     'ફ', 'બ', 'ભ', 'મ', 'ય', 'ર', 'લ', 'વ','શ','ષ' , 'સ' ,'હ','ળ','ક્ષ', 'જ્ઞ','0']
# Load Gujarati font
font_path = "NotoSerifGujarati-Regular.ttf"
font = ImageFont.truetype(font_path, 32)

# Preprocess image for model prediction
def preprocess_image(image_path):
    img_ = cv2.imread(image_path, cv2.IMREAD_ANYCOLOR)
    img_ = cv2.fastNlMeansDenoisingColored(img_, None, 3, 11, 7, 21)  # Denoise image
    img_ = cv2.resize(img_, (224, 224))  # Resize to match model's input shape
    img_array = img_to_array(img_)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Classify image and get predicted label
def classify_image(image_path):
    img_array = preprocess_image(image_path)
    prediction = model2.predict(img_array)
    # predicted_class = np.argmax(prediction)
    # predicted_label = class_labels[predicted_class]  
    return prediction

def print_label(prediction):
    dic = {}
    for i in range(len(class_labels)):
        dic[class_labels[i]] = prediction[0][i]
    out =list( sorted(dic.items(), key=lambda x: x[1],reverse=True))
    return out[:5]    

# Initialize time for label updates
last_pred_time = time.time()
predicted_label= list((list((" "," ")) , list((" ", " ")),))

while True:
    try:
        check, frame = webcam.read()
        
        if not check:
            break
        
        # Define the region of interest (ROI) and draw rectangle
        cv2.rectangle(frame, (region_left, region_top), (region_right, region_bottom), (0, 0, 0), 2)

        # Capture the region of interest (ROI)
        region = frame[region_top:region_bottom, region_left:region_right]
        
        if time.time() - last_pred_time >= 5:
            cv2.imwrite(filename='saved_img.jpg', img=region)
            input_img = Image.open("saved_img.jpg")
            output_img = remove(input_img)
            output_img.save("saved_img2.png", "PNG")
            prediction = classify_image('saved_img2.png')
            predicted_label = print_label(prediction)

            last_pred_time = time.time()

        # Convert OpenCV image to PIL for text rendering
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_image)

        # Render Gujarati text with PIL
        draw.text((region_left, region_top - 35), f"અનુમાનિત અક્ષર  : {predicted_label[0][0]} {predicted_label[1][0]}", font=font, fill=(0, 0, 0))
        # draw.text((region_left, region_bottom + 10), f"Predicted: {predicted_label2}", font=font, fill=(0, 0, 0))

        # Convert back to OpenCV format
        frame_with_text = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

        # Show the frame with prediction
        cv2.imshow("Capturing", frame_with_text)
 
        # Check if 'q' is pressed to exit
        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    except KeyboardInterrupt:
        print("Turning off camera.")
        break

# Release resources
webcam.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 664ms/step


In [ ]:
proper op =[થ ,દ ,ધ ,]